In [3]:
import json

In [13]:
import csv

In [27]:
!pip3 install lxml


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


Ran ```esearch -db pubmed -query "("intelligence"[Title/Abstract]) AND ((fha[Filter]) AND (2014:2024[pdat]))" | efetch -format xml > pubmed_results.xml``` in the command line, this saved all the data in pubmed_results.xml file

In [4]:
from lxml import etree
import json

def xml_to_dict(element):
    """
        Function to convert the XML data to a dictionary recursively
    """
    if len(element):
        result = {}
        for child in element:
            child_data = xml_to_dict(child)
            if child.tag in result:
                if type(result[child.tag]) is list:
                    result[child.tag].append(child_data)
                else:
                    result[child.tag] = [result[child.tag], child_data]
            else:
                result[child.tag] = child_data
        return result
    else:
        return element.text

def xml_to_json(xml_file_path, json_file_path):
    """
        Function to convert XML file to JSON file
    """
    # Parse the XML file
    parser = etree.XMLParser(recover=True)
    tree = etree.parse(xml_file_path, parser=parser)
    root = tree.getroot()

    # Convert XML to dictionary
    xml_dict = xml_to_dict(root)

    # Convert dictionary to JSON
    json_data = json.dumps(xml_dict, indent=2)

    # Write JSON data to a file
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)

xml_file_path = 'pubmed_results.xml' # input XML File
json_file_path = 'output.json' # Output JSON file

xml_to_json(xml_file_path, json_file_path)



In [5]:
import json

# Open the JSON file
with open('output.json', 'r') as file:
    # Load JSON data
    json_data = json.load(file)



In [6]:
# check how many articles we have
len(json_data["PubmedArticle"])

60503

In [16]:
def extract_required_keys(dictionary, extracted_data):
    """
        Function recursively checks the dictionary for the required keys and returns
        a new dictionary with only the extracted keys

        :param dictionary [type:Dict] - contains data pertaining to one PUBMED article
        :param extracted_data [type:Dict] - contains extracted data

        :return  extracted_data [type:Dict] - contains extracted data
    """
    
    for key, value in dictionary.items():
        if(key == "PMID"):
            extracted_data["PMID"] = value
        if(key == "ArticleTitle"):
            extracted_data["title"] = value
        if(key == "doi"):
            extracted_data["ELocationID"] = value
        if(key == "AbstractText"):
            extracted_data["text"] = value
        if(key == "AuthorList"):
            author_list = []
            for child in value["Author"]:
                if("ForeName" in child and "LastName" in child):
                    author_name=str(child["ForeName"])+" "+str(child["LastName"])
                    author_list.append(author_name)
            extracted_data["authors"] = author_list
        if isinstance(value, dict):
            extract_required_keys(value,extracted_data)
    return extracted_data

In [17]:
def writeDataToCSVFile(data, abstract_filename):
  """
    Function to write the extracted data to a CSV file
  """
  # name of csv file
  filename = abstract_filename
  #writing to csv file
  with open(filename, 'a') as csvfile:
    #creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the data rows
    csvwriter.writerows([[data["PMID"], data["doi"], data["authors"], data["title"], data["text"]]])

In [15]:
# Dictionary to store the desired information
result_dict = []
for entry in json_data["PubmedArticle"]:
    extracted_data= {
        "PMID":"",
      "doi":"",
      "authors":"",
      "title":"",
      "text":""
    }
    my_data = extract_required_keys(entry,extracted_data) # get the extracted data
    result_dict.append(my_data) # append extracted data to a local array for inspection
    writeDataToCSVFile(my_data,"extracted_outputs.csv") #apped extracted data to a CSV file

In [ ]:
len(result_dict)

60503

In [ ]:
result_dict[32]

{'PMID': '23480642',
 'title': 'The analysis of challenging relations: influences on interactive behaviour of staff towards clients with intellectual disabilities.',
 'text': ['Relationships between support staff and clients with intellectual disability (ID) are important for quality of care, especially when dealing with challenging behaviour. Building upon an interpersonal model, this study investigates the influence of client challenging behaviour, staff attitude and staff emotional intelligence on interactive behaviour of one of these relationship partners, being support staff.',
  'A total of 158 support staff members completed a questionnaire on staff interactive behaviour for 158 clients with ID and challenging behaviour, as well as two questionnaires on staff interpersonal attitude and emotional intelligence.',
  'Confronted with challenging behaviour as opposed to no challenging behaviour, staff reported less friendly, more assertive control and less support-seeking interperson

In [ ]:
result_dict[1]

{'PMID': '22442254',
 'title': 'Academic procrastination, emotional intelligence, academic self-efficacy, and GPA: a comparison between students with and without learning disabilities.',
 'text': "Academic procrastination has been seen as an impediment to students' academic success. Research findings suggest that it is related to lower levels of self-regulated learning and academic self-efficacy and associated with higher levels of anxiety, stress, and illness. Emotional intelligence (EI) is the ability to assess, regulate, and utilize emotions and has been found to be associated with academic self-efficacy and a variety of better outcomes, including academic performance. Students with learning disabilities (LD) are well acquainted with academic difficulty and maladaptive academic behavior. In comparison to students without LD, they exhibit high levels of learned helplessness, including diminished persistence, lower academic expectations, and negative affect. This study examined the re